In [17]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.impute import SimpleImputer
import seaborn as sns
sns.set() # Set searborn as default
from sklearn import model_selection
from sklearn import linear_model
import csv
import torch
import pyro
import pyro.distributions as dist

In [18]:
train_path = 'data/train_ver2.csv'
test_path = 'data/test_ver2.csv'
pd.set_option('display.max_columns', 50)


nrows_test=200
nrows_train=250
df_test = pd.read_csv(test_path ,na_values = ['?'],nrows=nrows_test)
df_train = pd.read_csv(train_path, sep=",",na_values = ['?'],nrows=nrows_train,engine='python') #nrows = 100, for loading fast



df_test['renta']=df_test['renta'].str.lstrip()
df_test['renta']=df_test['renta'].replace('NA', np.nan)


df_train['renta']=df_train['renta'].replace('     NA', np.nan)

#df_train['age']=df_train['age'].to_string()
#df_test['age']=df_test["age"].str.lstrip()
df_train['age']=df_train['age'].replace(' NA', np.nan)
 

print("Dataset, Nan values \n",df_train.isna().sum()/nrows_train)



Dataset, Nan values 
 fecha_dato               0.000
ncodpers                 0.000
ind_empleado             0.000
pais_residencia          0.000
sexo                     0.000
age                      0.000
fecha_alta               0.000
ind_nuevo                0.000
antiguedad               0.000
indrel                   0.000
ult_fec_cli_1t           1.000
indrel_1mes              0.000
tiprel_1mes              0.000
indresi                  0.000
indext                   0.000
conyuemp                 1.000
canal_entrada            0.000
indfall                  0.000
tipodom                  0.000
cod_prov                 0.000
nomprov                  0.000
ind_actividad_cliente    0.000
renta                    0.192
segmento                 0.000
ind_ahor_fin_ult1        0.000
ind_aval_fin_ult1        0.000
ind_cco_fin_ult1         0.000
ind_cder_fin_ult1        0.000
ind_cno_fin_ult1         0.000
ind_ctju_fin_ult1        0.000
ind_ctma_fin_ult1        0.000
ind_ctop_fin_ult1

In [19]:
columns_num= ['age','renta']
columns_cat= ['ind_empleado', 'ncodpers',  'indrel','pais_residencia', 'sexo','ind_nuevo', 'tiprel_1mes', 'indresi','indrel_1mes','indext','canal_entrada','conyuemp', 'indfall', 'nomprov', 'segmento','tipodom','cod_prov','ind_actividad_cliente']
columns_date=['fecha_dato','fecha_alta','ult_fec_cli_1t']


df_train[columns_cat]=df_train[columns_cat].astype('object')
df_train[columns_cat]=df_train[columns_cat].astype('object')
#df_train[columns_num]=df_train[columns_num].astype('int64')


In [20]:



print(df_train[['age','antiguedad']].describe())
print("\n \n  numerical value statistics \n \n", df_train[['renta']].describe())
print("\n \n  \n  categorical value statistics \n\n",df_train[columns_cat].describe())
print("\n \n  \n  date value statistics \n \n",df_train[columns_date].describe())




              age  antiguedad
count  250.000000  250.000000
mean    27.596000   34.880000
std      8.692007    1.834957
min     22.000000    6.000000
25%     23.000000   35.000000
50%     24.000000   35.000000
75%     29.000000   35.000000
max     67.000000   35.000000

 
  numerical value statistics 
 
                renta
count     202.000000
mean    99548.780941
std     67452.066713
min     13093.260000
25%     58952.437500
50%     84437.835000
75%    120053.752500
max    640168.260000

 
  
  categorical value statistics 

        ind_empleado  ncodpers  indrel pais_residencia sexo  ind_nuevo  \
count           250       250     250             250  250        250   
unique            1       250       1               1    2          1   
top               N   1050623       1              ES    H          0   
freq            250         1     250             250  132        250   

       tiprel_1mes indresi  indrel_1mes indext canal_entrada conyuemp indfall  \
count          250

In [21]:

#print("total number of columns",len(columns_num)+len(columns_cat)+len(columns_date))
df_test=df_test.drop(columns=['conyuemp'])
df_test=df_test.drop(columns=['ult_fec_cli_1t'])
df_train=df_train.drop(columns=['conyuemp'])
df_train=df_train.drop(columns=['ult_fec_cli_1t'])

columns_cat.remove('conyuemp')
columns_date.remove('ult_fec_cli_1t')


columns_x=df_train.columns[0:-24]
columns_y=df_train.columns[-24:]

df_train_x=df_train[columns_x]
df_train_y=df_train[columns_y]




print("Dataset, Nan values \n",df_train.isna().sum().sum())



def date_to_int(df, columns_date):
    
    imp_date = SimpleImputer(strategy="most_frequent")
    df[columns_date]=imp_date.fit_transform(df[columns_date])
   
    for col in columns_date:
        df[col]=df[col].str.replace("-","").astype(int)
    return df
    

def fill_NaN_values(df, columns_num,columns_cat,columns_date):
    
    imp_num = SimpleImputer(strategy="mean")
    df[columns_num]=imp_num.fit_transform(df[columns_num])
    
    imp_cat = SimpleImputer(strategy="most_frequent")
    df[columns_cat]=imp_cat.fit_transform(df[columns_cat])
    
    imp_date = SimpleImputer(strategy="most_frequent")
    df[columns_cat]=imp_date.fit_transform(df[columns_cat])
    
    return df
    

df_test = date_to_int(df_test,columns_date)
df_test = fill_NaN_values(df_test,columns_num,columns_cat,columns_date)

df_train = date_to_int(df_train, columns_date)
df_train = fill_NaN_values(df_train,columns_num,columns_cat,columns_date)


print("Dataset, Nan values \n",df_test.isna().sum().sum())



Dataset, Nan values 
 48
Dataset, Nan values 
 0


In [22]:
def one_hot_encodings(df):

    one_hot_encodings = {}
    for i in range(len(columns_cat)):
        df_dummies = pd.get_dummies(df[columns_cat[i]], prefix = columns_cat[i])
        one_hot_encodings[columns_cat[i]] = df_dummies

    # Drop original categorical columns
    df = df.drop(columns=list(columns_cat))

    # Append the one-hot encodings of the categorical columns
    for _, value in one_hot_encodings.items():
        df = pd.concat([df, value], axis=1)
        
    return df

df_test = one_hot_encodings(df_test)
df_train_x = one_hot_encodings(df_train_x)

df_test.head(5)

,fecha_dato,age,fecha_alta,antiguedad,renta,ind_empleado_F,ind_empleado_N,ncodpers_15889,ncodpers_1170280,ncodpers_1170281,ncodpers_1170283,ncodpers_1170285,ncodpers_1170286,ncodpers_1170288,ncodpers_1170289,ncodpers_1170290,ncodpers_1170293,ncodpers_1170295,ncodpers_1170296,ncodpers_1170298,ncodpers_1170300,ncodpers_1170302,ncodpers_1170304,ncodpers_1170305,ncodpers_1170306,...,cod_prov_17,cod_prov_18,cod_prov_22,cod_prov_23,cod_prov_25,cod_prov_26,cod_prov_27,cod_prov_28,cod_prov_29,cod_prov_30,cod_prov_31,cod_prov_32,cod_prov_33,cod_prov_35,cod_prov_37,cod_prov_39,cod_prov_41,cod_prov_44,cod_prov_45,cod_prov_46,cod_prov_47,cod_prov_50,cod_prov_52,ind_actividad_cliente_0,ind_actividad_cliente_1
0,20160628,56.0,19950116,256,326124.900000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,20160628,36.0,20130828,34,90637.641399,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,20160628,22.0,20130828,34,90637.641399,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,20160628,22.0,20130828,34,148402.980000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,20160628,22.0,20130828,34,106885.800000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
